# Test biologically plausible online PCA network
On simple gaussian and non-gaussian inputs. 

## Test case studied in Minden et al., 2018

Multivariate gaussian with random orthogonal transformation of a well-defined covariance matrix. $N$ dimensions, $K$ projections learnt. 

Two cases: $N=10$, $K=3$, and $N=100$, $K=10$

Two algorithms: inverse-free principal subspace projection (ifPSP) and PSP with complete inverse calculation. 

With each algorithm, the projection matrix to compute based on the learnt weights is different: for ifPSP, must use the approximate inverse of $L$, while for PSP, the exact inverse is used (because it is also used in the algorithm). 

### I would need the original code to compare
I can't reproduce the subspace alignment error results exactly. 
Not sure whether it comes from 
1. How I implemented the algorithm (but it seems not, because I get identical results with their Similarity Matching code from Giovannucci, Minden et al., 2018 and with my implementation of that algorithm. 
2. How I implemented the test input data itself, although intermediate checks seem OK. 
But these are small differences, overall, my implementation still gives the PCA decomposition of the input process, as it should, and is overall correct. 


In [ ]:
import numpy as np
import scipy as sp
import scipy.stats
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

if "../" not in sys.path:
    sys.path.insert(0, "../")

import multiprocessing
multiprocessing.set_start_method("fork")
from psutil import cpu_count
n_cpu = cpu_count(logical=False)

from utils.statistics import principal_component_analysis, seed_from_gen

In [ ]:
def compute_pca_meankept(samp, do_proj=False, vari_thresh=1.0, force_svd=False):
    """ Given an array of samples, compute the empirical covariance and
    diagonalize it to obtain the principal components and principal values,
    which are what is returned.

    If less than 10*d samples, take SVD of the sample matrix directly
    divided by 1/sqrt(N-1), because this amounts to eigendecomposition of
    the covariance matrix, but with better numerical stability and accuracy
    (but it's a lot slower).

    Args:
        samp (np.array): nxp matrix for n samples of p dimensions each.
            Pass the values of a dataframe for proper slicing.
        do_proj (bool): if True, also project the sample points
        vari_thresh (float in [0., 1.]): include principal components until
            a fraction vari_thresh of the total variance is explained.
        force_svd (bool): if True, use SVD of the data matrix directly.
    Returns:
        p_values (np.ndarray): 1d array of principal values, descending order.
        p_components (np.ndarray): 2d array of principal components.
            p_components[:, i] is the vector for p_values[i]
        samp_proj (np.ndarray): of shape (samp.shape[0], n_comp) where n_comp
            is the number of principal components needed to explain
            vari_thresh of the total variance.
    """
    # Few samples: use SVD on the de-meaned data directly.
    if force_svd or samp.shape[0] <= 10*samp.shape[1]:
        svd_res = np.linalg.svd(samp.T / np.sqrt(samp.shape[0] - 1))
        # U, Sigma, V. Better use transpose so small first dimension,
        # because higher accuracy in eigenvectors in U
        # Each column of U is an eigenvector of samp^T*samp/(N-1)
        p_components = svd_res[0]
        p_values = svd_res[1]**2  # Singular values are sqrt of eigenvalues

    # Many samples are available; use covariance then eigen decomposition
    else:
        covmat = np.dot(samp.T, samp) / (samp.shape[0] - 1)
        p_values, p_components = np.linalg.eigh(covmat)
        # Sort in decreasing order; eigh returns increasing order
        p_components = p_components[:, ::-1]
        p_values = p_values[::-1]

    if do_proj:
        vari_explained = 0.
        total_variance = np.sum(p_values)
        n_comp = 0
        while vari_explained < total_variance*vari_thresh:
            vari_explained += p_values[n_comp]
            n_comp += 1
        samp_proj = samp.dot(p_components[:, :n_comp])

    else:
        samp_proj = None

    return p_values, p_components, samp_proj

## Algorithm implementation
Embedded in an online simulation where random samples are successively presented. 

In [ ]:
# Alpha(t) for small N case
def alpha_t_smallN(t):
    return 10 / (250 + t)

# Alpha(t) for large N case
def alpha_t_largeN(t):
    return 1.1e-3 if t < 1e4 else 1.0e-4

In [ ]:
# Version that uses the approximate inverse L
def integrate_biopca_ifpsp(m_init, l_init, update_bk, bk_init, biopca_params, bk_params, 
                     tmax, dt, seed=None, noisetype="normal"):
    # Note: keep lambda matrix as 1d diagonal only, replace dot products by:
    # Lambda.dot(A): Lambda_ii applied to row i, replace by Lambda_diag[:, None]*A element-wise
    # A.dot(Lambda): Lambda_ii applied to column i, just A*Lambda broadcasts right
    n_neu = m_init.shape[0]  # Number of neurons N_I
    n_dim = m_init.shape[1]  # Number of input neurons N_D
    bk_vari_init, bk_vec_init = bk_init
    assert n_dim == bk_vec_init.shape[0], "Mismatch between dimension of m and background"
    mrate, lrate, lambda_diag = biopca_params
    # mrate is a callable: function of time. 

    rng = np.random.default_rng(seed=seed)
    tseries = np.arange(0, tmax, dt)

    # Containers for the solution over time
    m_series = np.zeros([tseries.shape[0], n_neu, n_dim])  # series of M^T (N_IxN_D)
    l_series = np.zeros([tseries.shape[0], n_neu, n_neu])  # series of L (N_IxN_I)
    cbar_series = np.zeros([tseries.shape[0], n_neu])  # series of projections
    bkvec_series = np.zeros([tseries.shape[0], n_dim])  # Input vecs, convenient to compute inhibited output

    ## Initialize running variables, separate from the containers above to avoid side effects.
    c = np.zeros(n_neu)  # un-inhibited neuron activities (before applying L)
    cbar = np.zeros(n_neu)  # inhibited neuron activities (after applying L)
    bk_vari = bk_vari_init.copy()
    bkvec = bk_vec_init.copy()
    mmat = m_init.copy()
    lmat = l_init.copy()
    
    # Inverse of diagonal of L is used a few times per iteration
    # Indices to access diagonal and off-diagonal elements of L
    # Will be used often, so prepare in advance. Replace dot product
    # with diagonal matrix by element-wise products.
    diag_idx = np.diag_indices(l_init.shape[0])
    inv_l_diag = 1.0 / l_init[diag_idx]  # 1d flattened diagonal
    # Use this difference the only time M_d is needed per iteration
    # Faster to re-invert inv_l_diag than to slice lmat again
    # l_offd = lmat - dflt(1.0 / inv_l_diag)
    newax = np.newaxis
    dflt = np.diagflat

    # Initialize neuron activity with m and background at time zero
    c = inv_l_diag*(mmat.dot(bkvec))  # L_d^(-1) M^T x_0
    # Lateral inhibition between neurons
    cbar = c - inv_l_diag*np.dot(lmat-dflt(1.0 / inv_l_diag), c)

    # Store back some initial values in containers
    cbar_series[0] = cbar
    m_series[0] = m_init
    l_series[0] = l_init
    bkvec_series[0] = bkvec

    # Generate N(0, 1) noise samples in advance
    if (tseries.shape[0]-1)*bk_vari.size > 1e7:
        raise ValueError("Too much memory needed; consider calling multiple times for shorter times")
    if noisetype == "normal":
        noises = rng.normal(0, 1, size=(tseries.shape[0]-1,*bk_vari.shape))
    elif noisetype == "uniform":
        noises = rng.random(size=(tseries.shape[0]-1, *bk_vari.shape))
    else:
        raise NotImplementedError("Noise option {} not implemented".format(noisetype))

    t = 0
    for k in range(0, len(tseries)-1):
        ### Online PCA weights
        # Synaptic plasticity: update mmat, lmat to k+1 based on cbar at k
        # Adding new length-1 axes is faster than np.outer for c.x^T, c.c^T
        mmat = mmat + dt * mrate(t) * (cbar[:, newax].dot(bkvec[newax, :]) - mmat)
        lmat = lmat + dt * mrate(t) * lrate * (cbar[:, newax].dot(cbar[newax, :])
                        - lambda_diag[:, newax] * lmat * lambda_diag)
        # Update too the variable saving the inverse of the diagonal of L
        inv_l_diag = 1.0 / lmat[diag_idx]
        
        t += dt

        # Update background to time k+1, to be used in next time step (k+1)
        bkvec, bk_vari = update_bk(bk_vari, bk_params, noises[k], dt)

        # Neural dynamics (two-step) at time k+1, to be used in next step
        # Not using Lambda[:, newax] trick here: this is a product with a vector M.x
        # so the result has to be a vector too. 
        c = inv_l_diag*(mmat.dot(bkvec))  # L_d^(-1) M^T x_0
        # Lateral inhibition between neurons
        cbar = c - inv_l_diag*np.dot(lmat - dflt(1.0/inv_l_diag), c)

        # Save current state
        knext = (k+1)
        m_series[knext] = mmat
        l_series[knext] = lmat
        bkvec_series[knext] = bkvec
        cbar_series[knext] = cbar  # Save activity of neurons at time k+1

    return tseries, bkvec_series, m_series, l_series, cbar_series


In [ ]:
# Version that numerically inverts matrix L: online PSP
def integrate_biopca_psp(m_init, l_init, update_bk, bk_init, biopca_params, bk_params, 
                     tmax, dt, seed=None, noisetype="normal"):
    # Note: keep lambda matrix as 1d diagonal only, replace dot products by:
    # Lambda.dot(A): Lambda_ii applied to row i, replace by Lambda_diag[:, None]*A element-wise
    # A.dot(Lambda): Lambda_ii applied to column i, just A*Lambda broadcasts right
    n_neu = m_init.shape[0]  # Number of neurons N_I
    n_dim = m_init.shape[1]  # Number of input neurons N_D
    bk_vari_init, bk_vec_init = bk_init
    assert n_dim == bk_vec_init.shape[0], "Mismatch between dimension of m and background"
    mrate, lrate, lambda_diag = biopca_params
    # mrate is a callable: function of time. 

    rng = np.random.default_rng(seed=seed)
    tseries = np.arange(0, tmax, dt)

    # Containers for the solution over time
    m_series = np.zeros([tseries.shape[0], n_neu, n_dim])  # series of M^T (N_IxN_D)
    l_series = np.zeros([tseries.shape[0], n_neu, n_neu])  # series of L (N_IxN_I)
    cbar_series = np.zeros([tseries.shape[0], n_neu])  # series of projections
    bkvec_series = np.zeros([tseries.shape[0], n_dim])  # Input vecs, convenient to compute inhibited output

    ## Initialize running variables, separate from the containers above to avoid side effects.
    c = np.zeros(n_neu)  # un-inhibited neuron activities (before applying L)
    cbar = np.zeros(n_neu)  # inhibited neuron activities (after applying L)
    bk_vari = bk_vari_init.copy()
    bkvec = bk_vec_init.copy()
    mmat = m_init.copy()
    lmat = l_init.copy()
    linv = np.linalg.inv(lmat)
    newax = np.newaxis

    # Store back some initial values in containers
    m_series[0] = m_init
    l_series[0] = l_init
    bkvec_series[0] = bkvec

    # Generate N(0, 1) noise samples in advance
    if (tseries.shape[0]-1)*bk_vari.size > 1e7:
        raise ValueError("Too much memory needed; consider calling multiple times for shorter times")
    if noisetype == "normal":
        noises = rng.normal(0, 1, size=(tseries.shape[0]-1,*bk_vari.shape))
    elif noisetype == "uniform":
        noises = rng.random(size=(tseries.shape[0]-1, *bk_vari.shape))
    else:
        raise NotImplementedError("Noise option {} not implemented".format(noisetype))

    t = 0
    for k in range(0, len(tseries)-1):
        ### Online PCA weights
        # Neural dynamics (two-step) at time k, to be used in this
        cbar = linv.dot(mmat).dot(bkvec)
        
        # Synaptic plasticity: update mmat, lmat to k+1 based on cbar at k
        mmat = mmat + dt * mrate(t) * (cbar[:, newax].dot(bkvec[newax, :]) - mmat)
        lmat = lmat + dt * mrate(t) * lrate * (cbar[:, newax].dot(cbar[newax, :])
                        - lambda_diag[:, newax] * lmat * lambda_diag)
        # Update too the variable saving the inverse of the diagonal of L
        linv = np.linalg.inv(lmat)


        # Update background to time k+1, to be used in next time step (k+1)
        bkvec, bk_vari = update_bk(bk_vari, bk_params, noises[k], dt) 
        t += dt
        
        # Save current state
        knext = (k+1)
        m_series[knext] = mmat
        l_series[knext] = lmat
        bkvec_series[knext] = bkvec
        cbar_series[k] = cbar  # Save activity of neurons at time k+1
    
    cbar = linv.dot(mmat).dot(bkvec)
    cbar_series[-1] = cbar
    return tseries, bkvec_series, m_series, l_series, cbar_series


## Define the input process

Following Minden et al., 2018, each sample $\vec{x}_t$ is drawn from a normal distribution with zero mean and covariance matrix

$$ G = R \tilde{G} R^T $$

where $\tilde{G}$ is diagonal and $R$ is a random orthogonal matrix, uniform in the Haar measure. Not sure what that means, but anyways, it is generated by taking the QR decomposition of a random matrix with standard normal elements. See https://nhigham.com/2020/04/22/what-is-a-random-orthogonal-matrix/. 

To match how background samples are generated in my code, I use the Cholesky decomposition trick: a vector $\vec{x}$ with covariance matrix G can be generated from standard normal samples in $\vec{u}$ by taking $\vec{x} = \Psi \vec{u}$, where $G = \Psi \Psi^T$ for some $\Psi$. So, the update of the background simply consists in taking pre-generated normal samples and applying $\Psi$ on them. 

Now, note that we can get something equivalent to $\Psi$ (although not lower triangular) by defining $\psi \psi^T = \tilde{G}$, such that $G = (R \psi)(R \psi)^T$ and we can take $\Psi = R \psi$. Since $\tilde{G}$ is diagonal, $\psi$ is just a diagonal matrix with standard deviations instead of variances on the diagonal. So, we get an acceptable $\Psi = R \tilde{G}^{1/2}$ directly from our random $R$ and from the standard deviations on the diagonal of $\tilde{G}^{1/2}$. 


In [ ]:
# Code to generate a multivariate sample from N(0, 1) samples (stdnorm_vec)
def update_mvnormal(bk_vari, bk_params, stdnorm_vec, dt):
    # bk_vari and dt are arguments for general compatibility: not used. 
    psi = bk_params[0]  # Only parameter is the Cholesky of the covariance! 
    return psi.dot(stdnorm_vec), stdnorm_vec

# Generate a random orthogonal matrix
def random_orthogonal_mat(n, rng):
    # Scipy or my version both give similar test results
    #return sp.stats.ortho_group.rvs(dim=n, size=1, random_state=rng)
    q, r = np.linalg.qr(rng.standard_normal(size=[n, n]), mode="complete")
    return q.dot(np.diagflat(np.sign(np.diagonal(r))))
    

In [ ]:
# Check whether our trick with $\Psi$ generates
# samples with the right distribution
# and that we really generate a random orthogonal $R$. 
def test_construct(std, r, psi, rng=None):
    if rng is None:
        rng = np.random.default_rng()
    # Check r is an orthogonal matrix: R^T R is the identity matrix
    n = std.shape[0]
    assert np.allclose(r.T.dot(r), np.eye(n)), "\n" + str(r)
    
    # Check that the covariance matrix constructed from psi 
    # and the intended matrix are the same
    cov_mat = psi.dot(psi.T)
    cov_mat_target = r.dot(std**2).dot(r.T)
    assert np.allclose(cov_mat, cov_mat_target), str(cov_mat)
    
    # Quick check: random samples have the desired covariance matrix G
    # Each column is a sample
    n_test_samp = int(1e6)
    test_samples = psi.dot(rng.standard_normal(size=[n, n_test_samp]))
    
    # Relative tolerance of 1/sqrt(n_samples)
    test_cov_mat = test_samples.dot(test_samples.T) / (n_test_samp - 1)
    
    assert np.allclose(cov_mat, test_cov_mat, 
                atol=4.0/np.sqrt(n_test_samp)), str(test_cov_mat) + "\n" + str(cov_mat)
    return True

## Define the output metric
Frobenius norm: sum of squared norm of all columns:

$$ ||A||^2 = \mathrm{Tr}\left[ A^T A \right] = \sum_{j} \vec{a}^T_j \vec{a}_j $$

where $\vec{a}_j$ is the $j$th column of matrix $A$. 

Subspace alignment error between $A$ (matrix) and $B$ (target): 
$$ E_B(A) = \min_Q \frac{||QA - B||^2}{||B||^2} $$ 

In [ ]:
def frobnorm(mat):
    """ Compute Frobenius norm of matrix A, 
    ||A||^2 = Tr(A^T A). """
    return np.trace(mat.T.dot(mat))

def subspace_align_error(mat, target):
    """ Compute min_Q ||Q.dot(mat) - target||^2 / ||target||^2. 
    The solution to that orthogonal Procrustes problem is 
        Q = U V^T, where USV^T is the SVD of target.dot(mat.T)
    according to Wikipedia, citing the solution of Schönemann, 1966. 
    (https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem)
    """
    # Solve Procrustes problem
    u, s, vh = np.linalg.svd(target.dot(mat.T))
    q = u.dot(vh)
    # Compute alignment error
    return frobnorm(q.dot(mat) - target) / frobnorm(target)

def find_nearest_idx(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
# Test the subspace alignment error metric. 
def test_error_metric(n, rng):
    for i in range(10):
        # Error should be zero when comparing matrices
        # that differ by an orthogonal matrix
        a = rng.standard_normal(size=[n,  n])
        rmat = random_orthogonal_mat(n, rng)
        b = rmat.dot(a)
        assert subspace_align_error(a, b) <= 1e-14
        # Change a column of a, check that the error between a and aprime
        # is the same whether rmat is applied or not, i.e. check that
        # we do optimize over Q properly in the subspace error. 
        aprime = a.copy()
        aprime[:, -1] = rng.standard_normal(size=n)
        errdiff = abs(subspace_align_error(a, aprime) 
                      - subspace_align_error(a, rmat.dot(aprime)))
        assert errdiff <= 1e-14
    return True

In [ ]:
# Given simulation results, compute the subspace alignment error after
# various numbers of iterations (at npts log-spaced iteration numbers)
def compute_error_series(ts, bkvecs, ms, ls, lamb_mat, inv_kind="exact", npts=21):
    """ inv_kind: either 'exact' or 'taylor', for PSP or ifPSP, respectively. """
    # Compute offline PCA solution
    nk = ls.shape[1]
    pvals, uk_offline, _ = compute_pca_meankept(bkvecs, do_proj=False)
    uk_offline = uk_offline[:, :nk]

    # Compute online PCA solution at a few different times
    chosen_times = np.logspace(2.0, np.log10(ms.shape[0]), npts)
    chosen_indices = np.asarray([find_nearest_idx(ts, t) for t in chosen_times])

    m_series_sub = ms[chosen_indices]
    l_series_sub = ls[chosen_indices]
    # See what happens with random matrices
    #m_series_sub = np.random.standard_normal(size=m_series_sub.shape)
    #l_series_sub = np.random.standard_normal(size=l_series_sub.shape)
    if inv_kind == "exact": 
        linv_series_sub = np.linalg.inv(l_series_sub)
    
    elif inv_kind == "taylor":
        linv_series_sub = np.zeros(l_series_sub.shape)
        for i in range(npts):
            ldinv = 1.0 / np.diagonal(l_series_sub[i])
            ldoff = l_series_sub[i] - np.diagflat(1.0 / ldinv)
            linv_series_sub[i] = ldinv[:, None] * (np.eye(nk) - ldoff*ldinv[None, :])
    else:
        raise ValueError("Unknown inverse kind: {}".format(inv_kind))

    # Compute the uk projector from M and L at each of those times
    # and the error compared to uk_offline
    uk_online_sub = np.zeros([len(chosen_indices)] + list(uk_offline.shape))
    pro_error = []
    for i in range(npts):
        uk_online_sub[i] = (1.0/lamb_mat[:, None] * linv_series_sub[i].dot(m_series_sub[i])).T
        pro_error.append(subspace_align_error(uk_online_sub[i], uk_offline))
    # Dot product of u_K^T u_K should be identity matrix
    # Check that the unscaled UK projector F = L^{-1} M has norms FF^T = Lambda^2, 
    # So that U_K itself has norm 1
    #print(uk_online_sub[-1].T.dot(uk_online_sub[-1]))
    return chosen_times, pro_error

## Reproduce test cases
Both algorithms, both dimensionalities, with the prescribed parameters. 

Report medians across 100 trials of randomly generated $R$ matrices -- will take some time to run!

In [ ]:
def run_onetest_twoalgos(init_m, init_l, update_fct, init_bk, biopca_params, back_params,  
                    duration, dt, seeds, psi_mat, noisetype, npts, tst):
    # Run simulation and compute error with ifPSP
    lambd = biopca_params[-1]
    res = integrate_biopca_ifpsp(init_m, init_l, update_fct, init_bk, biopca_params, back_params,  
                duration, dt, seed=seeds[0], noisetype=noisetype)
    tser, bkvecser, mser, lser, cbarser = res
    chosen_t, align_ifpsp = compute_error_series(tser, bkvecser, mser, lser, lambd, 
                                                  inv_kind="taylor", npts=npts)
    #print("ifPSP:\n", lser[-1])
    # Run simulation and compute error with PSP
    res = integrate_biopca_psp(init_m, init_l, update_fct, init_bk, biopca_params, back_params,  
                duration, dt, seed=seeds[1], noisetype=noisetype)
    tser, bkvecser, mser, lser, cbarser = res
    chosen_t, align_psp = compute_error_series(tser, bkvecser, mser, lser, lambd, 
                                                  inv_kind="exact", npts=npts)     
    #print("PSP:\n", lser[-1])
    
    # Check covariance matrix of background samples, make sure the input is what we think
    #cov_mat_samples = np.dot(bkvecser.T, bkvecser) / (tser.size - 1)
    #errmsg = "Problem with the empirical covariance of the generated samples"
    #assert np.allclose(cov_mat_samples, psi_mat.dot(psi_mat.T), atol=4e-2), errmsg
    
    print("Finished test ntst = {}".format(tst))
    
    return chosen_t, align_ifpsp, align_psp
    
    

# Run ntst tests of both algorithms (PSP and ifPSP) for a choice of N, K
# Keep the duration of each test to 1e4 by default, to make things quicker. 
def run_tests_nkchoice(ntst, nn, nk, gtilde, lambd, alphat, duration=1e4, tau=0.5, npts=21, rng=None):
    # Check a few things
    assert gtilde.shape[0] == nn
    assert lambd.shape[0] == nk
    
    if rng is None:
        rng = np.random.default_rng()
    
    # Initial matrices. M: normally-distributed, mean 0 and variance 1/N
    init_m = rng.standard_normal(size=[nk, nn]) / np.sqrt(nn)
    # L: identity matrix
    init_l = np.eye(nk)
    dt = 1.0
    
    # Algorithm parameters
    biopca_params = [alphat, 1.0/tau, lambd]
    
    # Containers for the error as a function of the number
    # of iterations, for each test run. 
    err_psp = np.zeros([ntst, npts])
    err_ifpsp = np.zeros([ntst, npts])
    
    # To generate a new seed for each simulation
    seed_sequence = np.random.SeedSequence(seed_from_gen(rng))
    
    for tst in range(ntst):
        # Generate a random orthogonal matrix R for this test run
        r_mat = random_orthogonal_mat(nn, rng)
        
        # Square root decomposition of the actual covariance matrix
        psi_mat = r_mat.dot(gtilde)
        if not test_construct(gtilde, r_mat, psi_mat, rng=rng):
            print(r"Problem with the $\Psi$ matrix trick!!!")

        # Background parameters with current psi_mat
        init_bk = [np.zeros(nn, dtype=bool), psi_mat.dot(rng.standard_normal(size=nn))]
        back_params = [psi_mat]
        
        # Get new seeds
        seeds_tst = seed_sequence.spawn(2)
        
        res = run_onetest_twoalgos(init_m, init_l, update_mvnormal, init_bk, biopca_params,
                        back_params, duration, dt, seeds_tst, psi_mat, "normal", npts, tst)
        chosen_t, err_ifpsp[tst], err_psp[tst] = res
        
    return chosen_t, err_ifpsp, err_psp

In [ ]:
## Parallelized version of the tests
# Run ntst tests of both algorithms (PSP and ifPSP) for a choice of N, K
# Keep the duration of each test to 1e4 by default, to make things quicker. 
def run_tests_nkchoice_parallel(ntst, nn, nk, gtilde, lambd, alphat, 
            duration=1e4, tau=0.5, npts=21, rng=None, njob=n_cpu):
    # Check a few things
    assert gtilde.shape[0] == nn
    assert lambd.shape[0] == nk
    
    if rng is None:
        rng = np.random.default_rng()
    
    # Initial matrices. M: normally-distributed, mean 0 and variance 1/N
    init_m = rng.standard_normal(size=[nk, nn]) / np.sqrt(nn)
    # L: identity matrix
    init_l = np.eye(nk)
    dt = 1.0
    
    # Algorithm parameters
    biopca_params = [alphat, 1.0/tau, lambd]
    
    # Containers for the error as a function of the number
    # of iterations, for each test run. 
    err_ifpsp = np.zeros([ntst, npts])
    err_psp = np.zeros([ntst, npts])
    
    # To generate a new seed for each simulation
    seed_sequence = np.random.SeedSequence(seed_from_gen(rng))
    
    # Pool of workers, to parallelize
    pool = multiprocessing.Pool(njob)
    res_objs = []
    
    for tst in range(ntst):
        # Generate a random orthogonal matrix R for this test run
        r_mat = random_orthogonal_mat(nn, rng)
        
        # Square root decomposition of the actual covariance matrix
        psi_mat = r_mat.dot(gtilde)
        if not test_construct(gtilde, r_mat, psi_mat, rng=rng):
            print(r"Problem with the $\Psi$ matrix trick!!!")

        # Background parameters with current psi_mat
        init_bk = [np.zeros(nn, dtype=bool), psi_mat.dot(rng.standard_normal(size=nn))]
        back_params = [psi_mat]
        
        # Get new seeds
        seeds_tst = seed_sequence.spawn(2)
        
        # Launch this test in parallel of others
        res = pool.apply_async(run_onetest_twoalgos, 
                            args=(init_m, init_l, update_mvnormal, init_bk, biopca_params,
                            back_params, duration, dt, seeds_tst, psi_mat, "normal", npts, tst)
                            )
        res_objs.append(res)
    
    # Get the results
    res_objs_finished = [a.get() for a in res_objs]
    
    # Don't forget to close the Pool! 
    pool.close()
    
    # Store results in appropriate containers
    chosen_t = res_objs_finished[0][0]
    for tst in range(ntst):
        err_ifpsp[tst] = res_objs_finished[tst][1]
        err_psp[tst] = res_objs_finished[tst][2]
        
    return chosen_t, err_ifpsp, err_psp

## Test runs for small N, K

In [ ]:
rgen = np.random.default_rng(seed=0x88977399c6600332f6b98129df9126c1)

# Dimensionalities
n_n = 10  # input
n_k = 3  # output

# Check alignment subspace metric
if test_error_metric(n_n, rgen):
    print("Subspace alignment metric seems to work")

# Standard deviation matrix diagonalized
gtilde_std = np.sqrt(np.diagflat([1.0, 0.75, 0.5] + [0.2]*(n_n - 3)))

# Lambda matrix
lambd_mat = np.asarray([1, 0.85, 0.7])

tau_const = 0.5  # Use value reported in the paper

In [ ]:
# Run tests
n_tests = 50
n_checkpts = 11
duration = 1e5
chosen_times, errors_ifpsp, errors_psp = run_tests_nkchoice_parallel(
            n_tests, n_n, n_k, gtilde_std, lambd_mat, alpha_t_smallN, 
            duration=duration, tau=tau_const, npts=n_checkpts, rng=rgen, njob=n_cpu)


In [ ]:
# Plot the median alignment error as a function of iteration number
mederr_ifpsp = np.median(errors_ifpsp, axis=0)
mederr_psp = np.median(errors_psp, axis=0)
maxerr_ifpsp, minerr_ifpsp = np.quantile(errors_ifpsp, q=[0.95, 0.1], axis=0)
maxerr_psp, minerr_psp = np.quantile(errors_psp, q=[0.95, 0.1], axis=0)
fig, ax = plt.subplots()
li1, = ax.plot(chosen_times, mederr_ifpsp, label="ifPSP")
ax.fill_between(chosen_times, minerr_ifpsp, maxerr_ifpsp, color=li1.get_color(), 
                alpha=0.3, label="ifPSP CI 90 %")
li2, = ax.plot(chosen_times, mederr_psp, label="PSP", ls="--")
ax.fill_between(chosen_times, minerr_psp, maxerr_psp, color=li2.get_color(), 
                alpha=0.3, label="PSP CI 90 %")
ax.set(xlabel="Iterations", ylabel="Subspace alignment error", 
       yscale="log", xscale="log")

# Add reported values
reported_errs_t = [1e3, 1e4, 1e5]
ax.plot(reported_errs_t, [2.1e-2, 1.5e-4, 1.7e-5], label="ifPSP (reported)", 
        marker="o", ls="None", ms=10, color=li1.get_color())
ax.plot(reported_errs_t, [1.9e-2, 4.1e-4, 5.5e-5], label="PSP (reported)", 
        marker="s", ls="None", ms=10, color=li2.get_color())
ax.legend()
fig.tight_layout()
#fig.savefig("../figures/tests/test_ifpsp_n10_k3_align_subspace_error.pdf", 
#           transparent="True", bbox_inches="tight")
plt.show()
plt.close()

## Test runs for large N, K

In [ ]:
# Dimensionalities
n_n2 = 100  # input
n_k2 = 10  # output

# Check alignment subspace metric
if test_error_metric(n_n2, rgen):
    print("Subspace alignment metric seems to work")

# Standard deviation matrix diagonalized
gtilde_std2 = np.sqrt(np.diagflat([1 - k/(2*(n_k2-1)) for k in range(10)] + [0.02]*(n_n2 - 10)))

# Lambda matrix
lambd_mat2 = np.asarray([1 - 3*k / (10*(n_k2-1)) for k in range(n_k2)])

tau_const2 = 0.5  # Use value reported in the paper

In [ ]:
# Run tests
n_tests = 50
n_checkpts = 11
duration = 1e5

chosen_times2, errors_ifpsp2, errors_psp2 = run_tests_nkchoice_parallel(
        n_tests, n_n2, n_k2, gtilde_std2, lambd_mat2, alpha_t_largeN, 
        duration=duration, tau=tau_const, npts=n_checkpts, rng=rgen, njob=n_cpu)

In [ ]:
# Plot the median alignment error as a function of iteration number
mederr_ifpsp2 = np.median(errors_ifpsp2, axis=0)
mederr_psp2 = np.median(errors_psp2, axis=0)
maxerr_ifpsp2, minerr_ifpsp2 = np.quantile(errors_ifpsp2, q=[0.95, 0.1], axis=0)
maxerr_psp2, minerr_psp2 = np.quantile(errors_psp2, q=[0.95, 0.1], axis=0)
fig, ax = plt.subplots()
li1, = ax.plot(chosen_times2, mederr_ifpsp2, label="ifPSP")
ax.fill_between(chosen_times2, minerr_ifpsp2, maxerr_ifpsp2, color=li1.get_color(), 
                alpha=0.3, label="ifPSP CI 90 %")
li2, = ax.plot(chosen_times2, mederr_psp2, label="PSP", ls="--")
ax.fill_between(chosen_times2, minerr_psp2, maxerr_psp2, color=li2.get_color(), 
                alpha=0.3, label="PSP CI 90 %")
ax.set(xlabel="Iterations", ylabel="Subspace alignment error", 
       yscale="log", xscale="log")

# Add reported values
reported_errs_t = [1e3, 1e4, 1e5]
ax.plot(reported_errs_t, [1.0, 3.1e-3, 5.4e-4], label="ifPSP (reported)", 
        marker="o", ls="None", ms=10, color=li1.get_color())
ax.plot(reported_errs_t, [1.3, 1.5e-3, 1.4e-4], label="PSP (reported)", 
        marker="s", ls="None", ms=10, color=li2.get_color())
ax.legend()
fig.tight_layout()
#fig.savefig("../figures/tests/test_ifpsp_n100_k10_align_subspace_error.pdf", 
#           transparent="True", bbox_inches="tight")
plt.show()
plt.close()

## Run one trial and visualize learnt components
Even if we are not reproducing the exact error metrics reported in the paper, are we at least capturing the right components?

Try a simple case with $N=2$, $K=2$ so that the full data can be plotted and the PCs overlaid on it. 

In [ ]:
def run_details_ifpsp(nn, nk, gtilde, lambd, alphat, duration=1e4, tau=0.5, rng=None):
    # Assuming gtilde is diagonal
    # Check a few things
    assert gtilde.shape[0] == nn
    assert lambd.shape[0] == nk
    
    if rng is None:
        rng = np.random.default_rng()
    
    # Initial matrices. M: normally-distributed, mean 0 and variance 1/N
    init_m = rng.standard_normal(size=[nk, nn]) / np.sqrt(nn)
    # L: identity matrix
    init_l = np.eye(nk)
    dt = 1.0
    
    # Rotation matrix transforming gtilde
    r_mat = random_orthogonal_mat(nn, rng)
    # Square root decomposition of the actual covariance matrix
    psi_mat = r_mat.dot(gtilde)
    if not test_construct(gtilde, r_mat, psi_mat, rng=rng):
        print(r"Problem with the $\Psi$ matrix trick!!!") 
    
    # Exact PCA: eigenvalue decomposition of psi psi^T, 
    # since we know the eigenvalues on the diagonal of G
    # the eigenvectors are columns in the r_mat
    #eigvals, eigvecs = np.linalg.eigh(psi_mat.dot(psi_mat.T))
    eigvecs = r_mat
    eigvals = np.diagonal(gtilde**2)
        
    # Algorithm parameters
    biopca_params = [alphat, 1.0/tau, lambd]

    # Background parameters with current psi_mat
    init_bk = [np.zeros(nn, dtype=bool), psi_mat.dot(rng.standard_normal(size=nn))]
    back_params = [psi_mat]
    
    # Run simulation and compute error with ifPSP
    res = integrate_biopca_ifpsp(init_m, init_l, update_mvnormal, init_bk, biopca_params, back_params,  
                duration, dt, seed=seed_from_gen(rng), noisetype="normal")
    tser, bkvecser, mser, lser, cbarser = res
    
    # Determine basis learnt by algorithm and return
    ldiag_inv = 1.0 / np.diagonal(lser[-1])
    ldoff = lser[-1] - np.diagflat(1.0 / ldiag_inv)
    linv = ldiag_inv[:, None] * (np.eye(nk) - ldoff*ldiag_inv[None, :])
    learntvecs = (1.0/lambd[:, None] * linv.dot(mser[-1])).T
    # Each column of learntvecs is an eigenvector learnt. 
    
    # Values on the diagonal of L are supposed to be eigenvalues
    learntvals = 1.0 / ldiag_inv
    
    # Compute
    
    return bkvecser, [eigvals, eigvecs], [learntvals, learntvecs], [tser, mser, lser, cbarser]

In [ ]:
# Dimensionalities
rgen_0 = np.random.default_rng(seed=0x2977c572e42f79c4c4b3f60de4687e08)
n_n0 = 3  # input
n_k0 = 2  # output

# Check alignment subspace metric
if test_error_metric(n_n0, rgen):
    print("Subspace alignment metric seems to work")

# Standard deviation matrix diagonalized
gtilde_std0 = np.sqrt(np.diagflat([1.0, 0.75, 0.5]))

# Lambda matrix
lambd_mat0 = np.asarray([1, 0.7])

tau_const = 0.5  # Use value reported in the paper

In [ ]:
bk_samples, true_pca, learnt_pca, learnser = run_details_ifpsp(n_n0, n_k0, gtilde_std0, lambd_mat0, 
                                                    alpha_t_smallN, duration=1e5, tau=tau_const, rng=rgen_0)

In [ ]:
print(learnser[2][-1])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.scatter(bk_samples[::10, 0], bk_samples[::10, 1], bk_samples[::10, 2], 
           color="grey", s=1, alpha=0.5, zorder=1)
scale = max(ax.get_xlim()[1], ax.get_ylim()[1], ax.get_zlim()[1])
for i in range(n_k0):
    ax.text(x=true_pca[1][0, i]*scale, y=true_pca[1][1, i]*scale, z=true_pca[1][2, i]*scale, 
            s="{:.2f}".format(true_pca[0][i]), fontsize=8, zorder=6+4*i, va="top")
    ax.quiver(0, 0, 0, *(true_pca[1][:, i]*scale), color="k", zorder=7+4*i)
    ax.text(x=learnt_pca[1][0, i]*scale, y=learnt_pca[1][1, i]*scale, z=learnt_pca[1][2, i]*scale, 
            s="{:.2f}".format(learnt_pca[0][i]), fontsize=8, ha="right", va="top", zorder=8+4*i)
    ax.quiver(0, 0, 0, *(scale*learnt_pca[1][:, i]), color="b", zorder=9+4*i)
ax.view_init(elev=50, azim=340)
ax.set(xlabel="x", ylabel="y", zlabel="z")
#fig.savefig("../figures/tests/online_pca_ifpsp_learnt_vectors_3d_example.pdf", 
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
chosen_t, align_psp = compute_error_series(learnser[0], bk_samples, learnser[1], learnser[2], lambd_mat0, 
                                                  inv_kind="taylor", npts=201)
fig, ax = plt.subplots()
ax.plot(chosen_t, align_psp)
ax.set(xlabel="Time (steps)", ylabel="Subspace alignment error", yscale="log", xscale="log")
plt.show()
plt.close()

# Online PSP version from another paper
Giovannucci, Minden et al., 2018. Code is available on Github for this one, so I can test even more directly their algorithm (copy-pasted their code). Slightly different version, using matrix inversion and no $\Lambda$ matrix (so we should compare with our code when setting $\Lambda = 1$. 

As it turns out, the algorithm is really the same as the ``biopca_inv`` that I tested above, with the exception of an added factor 2 in the $M$ learning rate. This is actually important, because Minden 2018 reports using $\tau < 1$, which seems to make the performance very poor; the factor $2$ added in Giovannucci and Minden compensates this difference and improves the error a lot. 

For an exact comparison of my PSP and theirs, I set $\tau$ twice as small ($\frac{1}{\tau}$) twice as large for their version, so the 2 in $M$'s equation is compensated in $L$'s equation by this $\tau$, and I set $\alpha_t$ twice as small to compensate the overall added factor of two. 

In [ ]:
def alpha_sm_smallN(t):
    return 5 / (250 + t)

def alpha_sm_largeN(t):
    return 5.5e-4 if t <= 1e4 else 5e-5

In [ ]:
# Copy-pasted code from 
# https://github.com/flatironinstitute/online_psp/blob/master/online_psp/similarity_matching.py
class SM:
    """
    Parameters:
    ====================
    K             -- Dimension of PCA subspace to learn
    D             -- Dimensionality of data
    M0            -- Initial guess for the lateral weight matrix M, must be of size K-by-K
    W0            -- Initial guess for the forward weight matrix W, must be of size K-by-D
    learning_rate -- Learning rate as a function of t
    tau           -- Learning rate factor for M (multiplier of the W learning rate)
    Methods:
    ========
    fit_next()
    """

    def __init__(self, K, D, M0=None, W0=None, learning_rate=alpha_sm_smallN, tau=0.25):

        if M0 is not None:
            assert M0.shape == (K, K), "The shape of the initial guess Minv0 must be (K,K)=(%d,%d)" % (K, K)
            M = M0
        else:
            M = np.eye(K)

        if W0 is not None:
            assert W0.shape == (K, D), "The shape of the initial guess W0 must be (K,D)=(%d,%d)" % (K, D)
            W = W0
        else:
            W = np.random.normal(0, 1.0 / np.sqrt(D), size=(K, D))


        self.eta = learning_rate
        self.t = 0

        self.K = K
        self.D = D
        self.tau = tau
        self.M = M
        self.W = W

        # Storage variables to allocate memory and optimize outer product time
        self.outer_W = np.empty_like(W)
        self.outer_M = np.empty_like(M)

    def fit_next(self, x):

        assert x.shape == (self.D,)

        t, tau, W, M, K = self.t, self.tau, self.W, self.M, self.K

        y = np.linalg.solve(M, W.dot(x))

        # Plasticity, using gradient ascent/descent
        # TODO: the factor of 2 can go away probably...
        # W <- W + 2 eta(t) * (y*x' - W)
        step = self.eta(t)

        np.outer(2 * step * y, x, self.outer_W)
        W = (1 - 2 * step) * W + self.outer_W

        # M <- M + eta(self.t)/tau * (y*y' - M)
        step = step / tau
        np.outer(step * y, y, self.outer_M)
        M = (1 - step) * M + self.outer_M

        self.M = M
        self.W = W

        self.t += 1

    def get_components(self, orthogonalize=True):
        '''
        Extract components from object
        orthogonalize: bool
            whether to orthogonalize the components before returning
        Returns
        -------
        components: ndarray
        '''

        components = np.asarray(np.linalg.solve(self.M, self.W).T)
        if orthogonalize:
            components, _ = np.linalg.qr(components)

        return components
    
# Using the class above, call fit_next repeatedly and save attributes
# each time. 
def integrate_opca_sm(m_init, l_init, update_bk, bk_init, opca_params, bk_params, 
                     tmax, dt, seed=None, noisetype="normal"):
    n_neu = m_init.shape[0]  # Number of neurons N_I
    n_dim = m_init.shape[1]  # Number of input neurons N_D
    bk_vari_init, bk_vec_init = bk_init
    assert n_dim == bk_vec_init.shape[0], "Mismatch between dimension of m and background"
    mrate, lrate = opca_params
    # mrate is a callable: function of time. 
    
    # Create relevant SM object
    smobj = SM(n_neu, n_dim, l_init, m_init, mrate, 1.0 / lrate)

    rng = np.random.default_rng(seed=seed)
    tseries = np.arange(0, tmax, dt)

    # Containers for the solution over time
    m_series = np.zeros([tseries.shape[0], n_neu, n_dim])  # series of M^T (N_IxN_D)
    l_series = np.zeros([tseries.shape[0], n_neu, n_neu])  # series of L (N_IxN_I)
    bkvec_series = np.zeros([tseries.shape[0], n_dim])  # Input vecs, convenient to compute inhibited output

    ## Initialize running variables, separate from the containers above to avoid side effects.
    bk_vari = bk_vari_init.copy()
    bkvec = bk_vec_init.copy()
    mmat = m_init.copy()
    lmat = l_init.copy()

    # Store back some initial values in containers
    m_series[0] = m_init
    l_series[0] = l_init
    bkvec_series[0] = bkvec

    # Generate N(0, 1) noise samples in advance
    if (tseries.shape[0]-1)*bk_vari.size > 1e7:
        raise ValueError("Too much memory needed; consider calling multiple times for shorter times")
    if noisetype == "normal":
        noises = rng.normal(0, 1, size=(tseries.shape[0]-1,*bk_vari.shape))
    elif noisetype == "uniform":
        noises = rng.random(size=(tseries.shape[0]-1, *bk_vari.shape))
    else:
        raise NotImplementedError("Noise option {} not implemented".format(noisetype))

    t = 0
    for k in range(0, len(tseries)-1):
        # Update synaptic weights with background from previous step
        smobj.fit_next(bkvec)
        
        # Update background to time k+1, to be used in next time step (k+1)
        bkvec, bk_vari = update_bk(bk_vari, bk_params, noises[k], dt)
        t += dt

        # Save current state
        knext = (k+1)
        m_series[knext] = smobj.W
        l_series[knext] = smobj.M
        bkvec_series[knext] = bkvec

    return tseries, bkvec_series, m_series, l_series, smobj


In [ ]:
# Run one test of the PSP algorithm obtained from Github
# And compare to my PSP algorithm
def run_onetest_sm_github(init_m, init_l, update_fct, init_bk, opca_params, biopca_params, 
                    back_params, duration, dt, seed, psi_mat, noisetype, npts, tst):
    lambd = biopca_params[-1]
    # Run simulation and compute error with SM PSP
    # m_init, l_init, update_bk, bk_init, opca_params, bk_params, 
                     #tmax, dt, seed=None, noisetype="normal"
    res = integrate_opca_sm(init_m, init_l, update_fct, init_bk, opca_params, back_params,  
                duration, dt, seed=seed, noisetype=noisetype)
    tser, bkvecser, mser, lser, cbarser = res
    chosen_t, align_smpsp = compute_error_series(tser, bkvecser, mser, lser, lambd, 
                                                  inv_kind="exact", npts=npts)

    # Run simulation and compute error with PSP
    res = integrate_biopca_psp(init_m, init_l, update_fct, init_bk, biopca_params, back_params,  
                duration, dt, seed=seed, noisetype=noisetype)
    tser, bkvecser, mser, lser, cbarser = res
    chosen_t, align_mypsp = compute_error_series(tser, bkvecser, mser, lser, lambd, 
                                                  inv_kind="exact", npts=npts)     

    # Check covariance matrix of background samples, make sure the input is what we think
    cov_mat_samples = np.dot(bkvecser.T, bkvecser) / (tser.size - 1)
    errmsg = "Problem with the empirical covariance of the generated samples"
    assert np.allclose(cov_mat_samples, psi_mat.dot(psi_mat.T), atol=4e-2), errmsg
    
    print("Finished test ntst = {}".format(tst))
    
    return chosen_t, align_smpsp, align_mypsp
    
    

# Run ntst tests of PSP algorithm obtained from Github
def run_tests_sm_github_parallel(ntst, nn, nk, gtilde, lambd, alphat_both, duration=1e4, 
                        tau=0.5, npts=21, rng=None, njob=n_cpu):
    # Check a few things
    assert gtilde.shape[0] == nn
    assert lambd.shape[0] == nk
    
    if rng is None:
        rng = np.random.default_rng()
    
    # Initial matrices. M: normally-distributed, mean 0 and variance 1/N
    init_m = rng.standard_normal(size=[nk, nn]) / np.sqrt(nn)
    # L: identity matrix
    init_l = np.eye(nk)
    dt = 1.0
    
    # Algorithm parameters
    # For Github version, alpha twice smaller, tau twice smaller. 
    opca_params = [alphat_both[0], 2.0/tau]
    biopca_params = [alphat_both[1], 1.0/tau, lambd]

    
    # Containers for the error as a function of the number
    # of iterations, for each test run. 
    err_smpsp = np.zeros([ntst, npts])
    err_mypsp = np.zeros([ntst, npts])
    
    # To generate a new seed for each simulation
    seed_sequence = np.random.SeedSequence(seed_from_gen(rng))
    
    # Pool of workers, to parallelize
    pool = multiprocessing.Pool(njob)
    res_objs = []
    
    for tst in range(ntst):
        # Generate a random orthogonal matrix R for this test run
        r_mat = random_orthogonal_mat(nn, rng)
        
        # Square root decomposition of the actual covariance matrix
        psi_mat = r_mat.dot(gtilde)
        if not test_construct(gtilde, r_mat, psi_mat, rng=rng):
            print(r"Problem with the $\Psi$ matrix trick!!!")

        # Background parameters with current psi_mat
        init_bk = [np.zeros(nn, dtype=bool), psi_mat.dot(rng.standard_normal(size=nn))]
        back_params = [psi_mat]
        
        # Get new seeds
        seed_tst = seed_sequence.spawn(1)
        
        # Launch this test in parallel of others
        res = pool.apply_async(run_onetest_sm_github, 
                            args=(init_m, init_l, update_mvnormal, init_bk, opca_params, biopca_params,
                            back_params, duration, dt, seed_tst[0], psi_mat, "normal", npts, tst)
                            )
        res_objs.append(res)
    
    # Get the results
    res_objs_finished = [a.get() for a in res_objs]
    
    # Don't forget to close the Pool! 
    pool.close()
    
    # Store results in appropriate containers
    chosen_t = res_objs_finished[0][0]
    for tst in range(ntst):
        err_smpsp[tst] = res_objs_finished[tst][1]
        err_mypsp[tst] = res_objs_finished[tst][2]
        
    return chosen_t, err_smpsp, err_mypsp

In [ ]:
# Run tests
n_tests = 50
n_checkpts = 11
duration = 1e5
lambd_mat = np.ones(n_k)

chosen_times, errors_smpsp, errors_mypsp = run_tests_sm_github_parallel(
        n_tests, n_n, n_k, gtilde_std, lambd_mat, [alpha_sm_smallN, alpha_t_smallN], 
        duration=duration, tau=tau_const, npts=n_checkpts, rng=rgen, njob=4)

In [ ]:
# Plot the median alignment error as a function of iteration number
mederr_smpsp = np.median(errors_smpsp, axis=0)
mederr_mypsp = np.median(errors_mypsp, axis=0)
fig, ax = plt.subplots()
li1, = ax.plot(chosen_times, mederr_smpsp, label="Github PSP")
li2, = ax.plot(chosen_times, mederr_mypsp, label="My PSP", ls="--")
ax.set(xlabel="Iterations", ylabel="Subspace alignment error", 
       yscale="log", xscale="log")

# Add reported values
reported_errs_t = [1e3, 1e4, 1e5]
#ax.plot(reported_errs_t, [2.1e-2, 1.5e-4, 1.7e-5], label="ifPSP (reported)", 
#        marker="o", ls="None", ms=10, color=li1.get_color())
ax.plot(reported_errs_t, [1.9e-2, 4.1e-4, 5.5e-5], label=r"PSP (reported, different $\mathbf{\Lambda}$)", 
        marker="s", ls="None", ms=6)
ax.legend()
fig.tight_layout()
#fig.savefig("../figures/tests/test_smpsp_n10_k3_align_subspace_error.pdf", 
#           transparent="True", bbox_inches="tight")
plt.show()
plt.close()

# Non-zero average inputs
For some reason, the online algorithm seems to fail. The outputs do not follow the properties derived in the paper (e.g. Lemma 3) even though the proofs should hold even when the average $\vec{x}$ is non-zero. 


In [ ]:
# Code to generate multivariate samples from N(0, 1) samples (stdnorm_vecs)
def generate_mvnormal_avg(bk_params, stdnorm_vecs):
    # stdnorm_vecs: indexed [dimension, samples]
    # psi: indexed [dimension, dimension]
    # Return: indexed [dimension, sample]
    psi = bk_params[0]
    avg = bk_params[1]
    return psi.dot(stdnorm_vecs) + avg[:, np.newaxis]

# Offline version of the algorithm, for comparison
Maybe the offline algorithm converges and the problem just comes from the online version being affected too much by a non-zero average $\vec{x}$. But I think both will fail if the online does.

In [ ]:
def offline_alpha(t):
    return 0.1

def taylor_inverse(invdiag, offdiag):
    return invdiag[:, np.newaxis] * (np.eye(invdiag.shape[0]) - offdiag*invdiag)

def integrate_offline_ifpsp(m_init, l_init, bk_samples, biopca_params, tmax):
    # Note: keep lambda matrix as 1d diagonal only, replace dot products by:
    # Lambda.dot(A): Lambda_ii applied to row i, replace by Lambda_diag[:, None]*A element-wise
    # A.dot(Lambda): Lambda_ii applied to column i, just A*Lambda broadcasts right
    n_neu = m_init.shape[0]  # Number of neurons N_I
    n_dim = m_init.shape[1]  # Number of input neurons N_D
    assert n_dim == bk_samples.shape[0], "Mismatch between dimension of m and background"
    mrate, lrate, lambda_diag = biopca_params
    # mrate is a callable: function of time. 

    tseries = np.arange(0, tmax, 1)

    # Containers for the solution over iterations
    m_series = np.zeros([tseries.shape[0], n_neu, n_dim])  # series of M^T (N_IxN_D)
    l_series = np.zeros([tseries.shape[0], n_neu, n_neu])  # series of L (N_IxN_I)
    f_series = np.zeros([tseries.shape[0], n_neu, n_dim])  # series of projectors F = L^{-1} M
    
    ## Initialize running variables
    xx_cov = bk_samples.dot(bk_samples.T) / (bk_samples.shape[1] - 1)
    yx_cov = np.zeros([n_neu, n_dim])  # F.dot(xx_cov) at every iteration
    yy_cov = np.zeros([n_neu, n_neu])  # F.dot(xx_cov).dot(F.T) at every iteration
    mmat = m_init.copy()
    lmat = l_init.copy()
    
    diag_idx = np.diag_indices(l_init.shape[0])
    inv_l_diag = 1.0 / l_init[diag_idx]  # 1d flattened diagonal
    # Use this difference the only time M_d is needed per iteration
    # Faster to re-invert inv_l_diag than to slice lmat again
    # l_offd = lmat - dflt(1.0 / inv_l_diag)
    newax = np.newaxis
    dflt = np.diagflat
    #fmat = np.linalg.inv(lmat).dot(mmat)
    fmat = taylor_inverse(inv_l_diag, lmat - dflt(1.0 / inv_l_diag)).dot(mmat)
    
    # Store back some initial values in containers
    m_series[0] = m_init
    l_series[0] = l_init
    f_series[0] = fmat

    for k in range(0, len(tseries)-1):
        ### Neural activity, averages over data given current projector. 
        yx_cov = fmat.dot(xx_cov)
        yy_cov = yx_cov.dot(fmat.T)
        
        ### Synaptic plasticity: update mmat, lmat to k+1 based on covariances at k
        mmat = mmat + mrate(k) * (yx_cov - mmat)
        lmat = lmat + mrate(k) * lrate * (yy_cov - lambda_diag[:, newax]*lmat*lambda_diag)
        # Update too the variable saving the inverse of the diagonal of L
        inv_l_diag = 1.0 / lmat[diag_idx]
        # and the projector F
        fmat = taylor_inverse(inv_l_diag, lmat - dflt(1.0 / inv_l_diag)).dot(mmat)
        #fmat = np.linalg.inv(lmat).dot(mmat)
        
        # Save current state
        knext = (k+1)
        m_series[knext] = mmat
        l_series[knext] = lmat
        f_series[knext] = fmat
    
    return tseries, m_series, l_series, f_series


In [ ]:
def integrate_offline_psp(m_init, l_init, bk_samples, biopca_params, tmax):
    # Note: keep lambda matrix as 1d diagonal only, replace dot products by:
    # Lambda.dot(A): Lambda_ii applied to row i, replace by Lambda_diag[:, None]*A element-wise
    # A.dot(Lambda): Lambda_ii applied to column i, just A*Lambda broadcasts right
    n_neu = m_init.shape[0]  # Number of neurons N_I
    n_dim = m_init.shape[1]  # Number of input neurons N_D
    assert n_dim == bk_samples.shape[0], "Mismatch between dimension of m and background"
    mrate, lrate, lambda_diag = biopca_params
    # mrate is a callable: function of time. 

    tseries = np.arange(0, tmax, 1)

    # Containers for the solution over iterations
    m_series = np.zeros([tseries.shape[0], n_neu, n_dim])  # series of M^T (N_IxN_D)
    l_series = np.zeros([tseries.shape[0], n_neu, n_neu])  # series of L (N_IxN_I)
    f_series = np.zeros([tseries.shape[0], n_neu, n_dim])  # series of projectors F = L^{-1} M
    
    ## Initialize running variables
    xx_cov = bk_samples.dot(bk_samples.T) / (bk_samples.shape[1] - 1)
    yx_cov = np.zeros([n_neu, n_dim])  # F.dot(xx_cov) at every iteration
    yy_cov = np.zeros([n_neu, n_neu])  # F.dot(xx_cov).dot(F.T) at every iteration
    mmat = m_init.copy()
    lmat = l_init.copy()
    
    newax = np.newaxis
    dflt = np.diagflat
    fmat = np.linalg.inv(lmat).dot(mmat)
    
    # Store back some initial values in containers
    m_series[0] = m_init
    l_series[0] = l_init
    f_series[0] = fmat

    for k in range(0, len(tseries)-1):
        ### Neural activity, averages over data given current projector. 
        yx_cov = fmat.dot(xx_cov)
        yy_cov = yx_cov.dot(fmat.T)
        
        ### Synaptic plasticity: update mmat, lmat to k+1 based on covariances at k
        mmat = mmat + mrate(k) * (yx_cov - mmat)
        lmat = lmat + mrate(k) * lrate * (yy_cov - lambda_diag[:, newax]*lmat*lambda_diag)
        # and the projector F
        fmat = np.linalg.inv(lmat).dot(mmat)
        
        # Save current state
        knext = (k+1)
        m_series[knext] = mmat
        l_series[knext] = lmat
        f_series[knext] = fmat
    
    return tseries, m_series, l_series, f_series

In [ ]:
def cov_product(a):
    return a.dot(a.T)

def run_offline_psp(algo, nn, nk, gtilde, lambd, alphat, nsamples=int(1e5), 
                      niter=int(1e4), tau=0.5, rng=None):
    """ algo is either integrate_offline_ifpsp or integrate_offline_psp"""
    # Assuming gtilde is diagonal
    # Check a few things
    assert gtilde.shape[0] == nn
    assert lambd.shape[0] == nk
    
    if rng is None:
        rng = np.random.default_rng()
    
    # Initial matrices. M: normally-distributed, mean 0 and variance 1/N
    init_m = rng.standard_normal(size=[nk, nn]) / np.sqrt(nn)
    # L: identity matrix
    init_l = np.eye(nk)
    dt = 1.0
    
    # Rotation matrix transforming gtilde
    r_mat = random_orthogonal_mat(nn, rng)
    # Square root decomposition of the actual covariance matrix
    psi_mat = r_mat.dot(gtilde)
    if not test_construct(gtilde, r_mat, psi_mat, rng=rng):
        print(r"Problem with the $\Psi$ matrix trick!!!") 
    
    # Average of the input samples
    avg_vec = 0*rng.random(size=nn)
    
    # Generate samples in advance
    bk_samples_offline = generate_mvnormal_avg([psi_mat, avg_vec], 
                            rng.standard_normal(size=[nn, nsamples]))

    # Exact PCA: eigenvalue decomposition of xx^T / (n_samples-1)
    eigvals, eigvecs = np.linalg.eigh(bk_samples_offline.dot(bk_samples_offline.T) / (nsamples-1))
    eigvals = eigvals[::-1]  # Put largest eigenvalues first
    eigvecs = eigvecs[:, ::-1]
        
    # Algorithm parameters
    biopca_params = [alphat, 1.0/tau, lambd]
    
    # Run simulation and compute error with ifPSP
    res = algo(init_m, init_l, bk_samples_offline, biopca_params, niter)
    tser, mser, lser, fser = res
    #print(cov_product(1.0/lambd[:, None] * fser[-1]))
    
    # Determine basis learnt by algorithm and return
    learntvecs = ((1.0/lambd[:, None]) * fser[-1]).T
    # Each column of learntvecs is an eigenvector learnt. 
    
    # Values on the diagonal of L are supposed to be eigenvalues of XX^T / nsamples
    learntvals = np.diagonal(lser[-1])
    print(lser[-1])
    
    return bk_samples_offline, [eigvals, eigvecs], [learntvals, learntvecs], [mser, lser, fser]

### Offline ifPSP
The offline version seems to always converge to all "eigenvalues" on the $L$ matrix diagonal being equal. 

Note that this is for zero average background; when the background average is non-zero, the eigenvalues are just crazy numbers. 

In [ ]:
# Dimensionalities
rgen_off = np.random.default_rng(seed=0xb61d55bd547e030074bdfaefaf1fad45)
n_n_off = 10  # input
n_k_off = 3  # output

# Check alignment subspace metric
if test_error_metric(n_n_off, rgen_off):
    print("Subspace alignment metric seems to work")

# Standard deviation matrix diagonalized
gtilde_std_off = np.sqrt(np.diagflat([1.0, 0.75, 0.5] + [0.2] * (n_n_off - 3)))

# Lambda matrix
#lambd_mat_off = np.asarray([1, 0.85])
lambd_mat_off = np.asarray([1, 0.85, 0.7])

tau_const = 1.0  # Use value reported in the paper

In [ ]:
res = run_offline_psp(integrate_offline_ifpsp, n_n_off, n_k_off, gtilde_std_off, lambd_mat_off, 
                offline_alpha, nsamples=int(1e5), niter=int(1e4), tau=tau_const, rng=rgen_off)
bk_samples_off, true_pca_off, learnt_pca_off, learn_series = res

In [ ]:
print("Offline ifPSP")
print("Subspace align. error:", subspace_align_error(learnt_pca_off[1], true_pca_off[1][:, :n_k_off]))

print("Learnt eigenvalues:", learnt_pca_off[0])
print("True eigenvalues:", true_pca_off[0][:n_k_off])
#print("Learnt eigenvectors (columns):\n", learnt_pca_off[1])
#print("True eigenvectors (columns):\n", true_pca_off[1][:, :n_k_off])

In [ ]:
# Time course of l diagonal
learntvalser = np.diagonal(learn_series[1], axis1=1, axis2=2)
sort_learnt_pca = np.argsort(learntvalser[-1])[::-1]
fig, axes = plt.subplots(2, 1, sharex=True)
ax = axes[0]
for i in range(learntvalser.shape[1]):
    li, = ax.plot(np.arange(1, learntvalser.shape[0]), learntvalser[1:, sort_learnt_pca[i]], 
                  label="Diagonal element ({},{})".format(i, i), lw=2.0)
    ax.axhline(true_pca_off[0][i], color=li.get_color(), ls="--", lw=1.0)
ax.set(ylabel=r"Diagonal elements of $L$", xscale="log")
ax.annotate("Principal values learning", xy=(0.99, 0.98), ha="right", va="top", xycoords="axes fraction")

# Subspace error
log_times = np.logspace(0, 4, 51).astype(int) - 1
log_times = log_times.clip(1)
learnt_pca_ser = [((1.0/lambd_mat_off[:, None]) * learn_series[2][i]).T for i in log_times]
error_series = [subspace_align_error(true_pca_off[1][:, :n_k_off], v) for v in learnt_pca_ser]
ax = axes[1]
ax.plot(log_times, error_series, color="k")
ax.set(xscale="log", xlabel="Iterations", ylabel="Subspace alignment error", yscale="log")
ax.annotate("Principal components learning", xy=(0.99, 0.98), xycoords="axes fraction", ha="right", va="top")
plt.show()
plt.close()

### Offline PSP 
With exact inverse, we sometimes get long-term convergence to distinct eigenvalues, sometimes to same eigenvalue. Seems completely degenerate. 

In [ ]:
rgen_off2 = np.random.default_rng(seed=0x839dcd7e6051b2b4b34d7785d4c5575e)
res = run_offline_psp(integrate_offline_psp, n_n_off, n_k_off, gtilde_std_off, lambd_mat_off, 
                offline_alpha, nsamples=int(1e5), niter=int(1e4), tau=tau_const, rng=rgen_off2)
bk_samples_off2, true_pca_off2, learnt_pca_off2, learn_series2 = res

In [ ]:
print("Offline PSP")
print("Subspace align. error:", subspace_align_error(learnt_pca_off2[1], true_pca_off2[1][:, :n_k_off]))

print("Learnt eigenvalues:", learnt_pca_off2[0])
print("True eigenvalues:", true_pca_off2[0][:n_k_off])
#print("Learnt eigenvectors (columns):\n", learnt_pca_off2[1])
#print("True eigenvectors (columns):\n", true_pca_off2[1][:, :n_k_off])

In [ ]:
# Time course of l diagonal
learntvalser = np.diagonal(learn_series2[1], axis1=1, axis2=2)
sort_learnt_pca = np.argsort(learntvalser[-1])[::-1]
fig, axes = plt.subplots(2, 1, sharex=True)
ax = axes[0]
for i in range(learntvalser.shape[1]):
    li, = ax.plot(np.arange(1, learntvalser.shape[0]), learntvalser[1:, sort_learnt_pca[i]], 
                  label="Diagonal element ({},{})".format(i, i), lw=2.0)
    ax.axhline(true_pca_off2[0][i], color=li.get_color(), ls="--", lw=1.0)
ax.set(ylabel=r"Diagonal elements of $L$", xscale="log")
ax.annotate("Principal values learning", xy=(0.99, 0.98), ha="right", va="top", xycoords="axes fraction")

# Subspace error
log_times = np.logspace(0, 4, 51).astype(int) - 1
log_times = log_times.clip(1)
learnt_pca_ser = [((1.0/lambd_mat_off[:, None]) * learn_series2[2][i]).T for i in log_times]
error_series = [subspace_align_error(true_pca_off2[1][:, :n_k_off], v) for v in learnt_pca_ser]
ax = axes[1]
ax.plot(log_times, error_series, color="k")
ax.set(xscale="log", xlabel="Iterations", ylabel="Subspace alignment error", yscale="log")
ax.annotate("Principal components learning", xy=(0.99, 0.98), ha="right", va="top", xycoords="axes fraction")
plt.show()
plt.close()